# Read the helpr class file 'All_helper_classes.ipynb' and instantiate the three helper classes

In [2]:
%run "All_helper_classes.ipynb" sort=False

#from "Models training helpers.ipynb" we will import 3 helper classes and the associated helper methods.
parsing_helpers = generic_parsing_helpers()
aivm_helper = AIVM_helper()
model_helpers = default_model_helpers_for_project()

Necessary libraries imported.
Initialised generic_parsing_helpers class and methods
Initialised AIVM_helper class and methods
Initialised model_helpers class and methods


In [3]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Get project paths from helper classes
access using paths.THE_PATH

In [4]:
gc.collect()
paths = project_paths()

# We are always reading the glove embeddings from the local GLOVE_DIR with specified dimensions. These embeddings are too large to store on Git_hub, stop words are also read locally for now


In [6]:
print ("paths.TRAINING_DATA_DIR_FILE_LOCAL: ", paths.TRAINING_DATA_DIR_FILE_LOCAL)
print ("paths.STOP_WORDS_PATH: ", paths.STOP_WORDS_PATH)
print ("paths.GLOVE_DIR_LOCAL: ", paths.GLOVE_DIR_LOCAL)
print ("model_helpers.EMBEDDING_DIM: ", model_helpers.EMBEDDING_DIM)

paths.TRAINING_DATA_DIR_FILE_LOCAL:  Data/Iterative-models-building/Training data/Conventions/training_aggregated_conventions.tsv
paths.STOP_WORDS_PATH:  Data/Iterative-models-building/Training data/resources/stopwords.txt
paths.GLOVE_DIR_LOCAL:  Data/Iterative-models-building/Training data/
model_helpers.EMBEDDING_DIM:  100


In [ ]:
#******************THIS CODE WAS MOVED BY AF, NEED TO ASK DAVID WHY IT EXISTS***************

# Read aggregated training data from TRAINING_DATA_DIR_FILE_LOCAL or TRAINING_DATA_DIR_FILE_GIT 
# and stores the content in a dataframe variable named 'training_df'. 
# Files in GATHERED_DATA_FOLDER are combined with the content of 'training_df' 
# to create the vocabulary of the Tokenizer, as by design, ```"tokenizer.tokenize()"``` 
# will remove any words not seen on the ```"tokenizer.fit()"``` stage.

#TODO we need to read some of this data directly from GIT_HUB not from local.

gathered_dfs = []

# We are always reading the glove embeddings from the local GLOVE_DIR with specified dimensions
# These embeddings are too large to store on Git_hub
embeddings_index = parsing_helpers.read_glove_embeddings(paths.GLOVE_DIR_LOCAL, model_helpers.EMBEDDING_DIM)
stopwords= parsing_helpers.get_stop_words(paths.STOP_WORDS_PATH)
                                                      
column_to_clean = 'text'    
training_df = parsing_helpers.get_clean_df_text_column(paths.TRAINING_DATA_DIR_FILE, column_to_clean)

gathered_data_files = [f for f in os.listdir(paths.GATHERED_CONVENTIONS_DATA_FOLDER)]
display ("all files in dir", gathered_data_files)
gathered_data_files = aivm_helper.remove_excluded_files(gathered_data_files) 
display ("only relevant files included",gathered_data_files)

for f_name in gathered_data_files:
    tmp_df = pd.read_csv(os.path.join(paths.GATHERED_CONVENTIONS_DATA_FOLDER, f_name), sep="\t")
    tmp_df = tmp_df.rename(columns={"sentence": "text"})
    gathered_dfs.append(tmp_df)

gathered_dfs.append(training_df)##Adding training data sentences

column_to_tokenize = 'text'
extended_tokenizer = parsing_helpers.create_tokenizer(pd.concat(gathered_dfs, sort=True), column_to_tokenize, max_words=MAX_NB_WORDS)

In [ ]:
EXECUTE_EXAMPLE = True
if EXECUTE_EXAMPLE: #### JUST TO AVOID EXECUTING THIS CELL EVERY TIME

    NUM_SENTENCES_TO_SHOW=10
    #################################################
    ## Example of how to use the models helper files
    #################################################
    display (training_df.head(3))
    ## This is done as example, test dataframe should be readed from a diferent file!
    test_df = training_df

    ## Tokenizer was already trained!
    assert(extended_tokenizer is not None)


    ## Train DL models

    _DLModels, _DLTokenizers, _DLData_val_x, _DLData_val_y, _DLTrain_histories = model_helpers.train_DL_models(df_train,
                    data_class_column="convention", 
                    data_label_column="label",
                    tokenizer=extended_tokenizer,
                    random_seed=0,
                    use_validation=False) 

    ## Train ML models
    _MLModels = model_helpers.train_ML_models(df_train) 

    ##Getting sentences from data
    test_sentences = test_df['text'].values

    ## Calculate probability scores for for text data combining DL predictions and ML predictions
    mixture_preds = model_helpers.calculate_matches_mixture(test_sentences, _MLModels, _DLModels, _DLTokenizers)

    ## Getting top sentences per convention
    for conv in _MLModels.keys():
        print("\n\n     {}    \n\n".format(conv))
        for s in test_sentences[mixture_preds[conv].argsort()[-NUM_SENTENCES_TO_SHOW:][::-1]]:
            print("{}\n".format(test_sentences[s]))
            
            
    ## Store DL models in a pickle file
    model_helpers.store_DL_models_in_picke("test.pickle", _DLModels, _DLTokenizers, _DLData_val_x, _DLData_val_y, _DLTrain_histories, paths.PICKLED_MODELS_DIR)
    
    ## Read DL models from a pickle file
    _DLModels, _DLTokenizers, _DLData_val_x, _DLData_val_y, _DLTrain_histories = model_helpers.read_pickle("test.pickle")


In [ ]:
CONVENTION = "Green"
df = training_df[training_df['convention'] == CONVENTION]

In [ ]:
## Checking performance setting validation data on the model training
## Validation data is created inside the models trainign function
conv_models = model_helpers.train_DL_models(df,
            data_class_column="convention", 
            data_label_column="label",
            df_val=None,
            tokenizer=extended_tokenizer,
            random_seed=0, 
            use_validation=True)

In [ ]:
## Checking performance setting validation data on the model training
## Validation data is created outside the models trainign function
## Training models
texts = df['text'].values
labels = df['label'].values

indices = np.arange(len(texts))
np.random.seed(0)
np.random.shuffle(indices)
texts = texts[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * len(texts))

x_train = texts[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = texts[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

df_train = pd.DataFrame(columns=['text', 'label', 'convention'])
df_train['text'] = x_train
df_train['label'] = y_train
df_train['convention'] = CONVENTION


df_val = pd.DataFrame(columns=['text', 'label', 'convention'])
df_val['text'] = x_val
df_val['label'] = y_val
df_val['convention'] = CONVENTION

print("\n\n===========================")
print("WITH VALIDATION!")
print("===========================\n\n")
models2 = model_helpers.train_DL_models(df_train,
                data_class_column="convention", 
                data_label_column="label",
                df_val=df_val,
                tokenizer=extended_tokenizer,
                random_seed=0, use_validation=True)


In [ ]:
######################
### Cross validation
######################

## 1. Split training, validation data outside the training function
## 2. Train a model with training data
## 3. Check performance (acc) of the model on training and validation
## 4. Repeat num_tests times
## Next cell plots results

## Number of tests = 10
num_tests = 10
num_epochs = 20

accs = []
val_accs = []
epochs = np.arange(0,num_epochs, 1)

texts = df['text'].values
labels = df['label'].values


for n in range(num_tests):
    print("\n\n=================================")
    print("               {}".format(n))
    print("=================================")
    indices = np.arange(len(texts))
    np.random.shuffle(indices)
    texts = texts[indices]
    labels = labels[indices]
    nb_validation_samples = int(VALIDATION_SPLIT * len(texts))

    x_train = texts[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    x_val = texts[-nb_validation_samples:]
    y_val = labels[-nb_validation_samples:]

    df_train = pd.DataFrame(columns=['text', 'label', 'convention'])
    df_train['text'] = x_train
    df_train['label'] = y_train
    df_train['convention'] = CONVENTION


    df_val = pd.DataFrame(columns=['text', 'label', 'convention'])
    df_val['text'] = x_val
    df_val['label'] = y_val
    df_val['convention'] = CONVENTION


    tmp_training = model_helpers.train_DL_models(df_train,
                data_class_column="convention", 
                data_label_column="label",
                df_val=df_val,
                tokenizer=extended_tokenizer,
                random_seed=0,
                use_validation=True, 
                num_epochs = num_epochs)

    accs.append(tmp_training[4][CONVENTION].history['acc'])
    val_accs.append(tmp_training[4][CONVENTION].history['val_acc'])       


In [ ]:
fig = plt.figure(figsize=[10,10])
fig.suptitle("Checking performance on multiple random data partitions\n       ({} Convention)".format(CONVENTION), fontsize=16)
plt.subplot(2, 2, 1)
plt.subplots_adjust(wspace=0.4, hspace=0.4)


for i in range(len(accs)):
    plt.plot(epochs,accs[i], label="Test {}".format(i))
plt.title("Training accuracies per epoch")
plt.xlabel("epochs")
plt.ylabel("acc score")
plt.legend()

plt.subplot(2, 2, 2)
for i in range(len(val_accs)):
    plt.plot(epochs,val_accs[i], label="Test {}".format(i))
plt.title("Validation accuracies per epoch")
plt.xlabel("epochs")
plt.ylabel("acc score")
plt.legend()
plt.show()

In [ ]:
models2_tokenizer = models2[1][CONVENTION]
val_seq = models2_tokenizer.texts_to_sequences(x_val)
val_seq = pad_sequences(val_seq, maxlen=MAX_SEQUENCE_LENGTH)

preds2 = models2[0][CONVENTION].predict(val_seq)
print("Real label for samples classified with value 0")
display(y_val[preds2[:,0]>preds2[:,1]])
print("Real label for samples classified with value 1")
display(y_val[preds2[:,0]<preds2[:,1]])

In [ ]:

print("\n\n===========================")    
print("WITHOUT VALIDATION!")
print("===========================\n\n")
models3 = model_helpers.train_DL_models(df_train,
                data_class_column="convention", 
                data_label_column="label",
                df_val=None,
                tokenizer = models2_tokenizer,
                random_seed=0, use_validation=False)


In [ ]:

val_seq = models2_tokenizer.texts_to_sequences(x_val)
val_seq = pad_sequences(val_seq, maxlen=MAX_SEQUENCE_LENGTH)

preds2 = models2[0][CONVENTION].predict(val_seq)
print("Real label for samples classified with value 0")
display(y_val[preds2[:,0]>preds2[:,1]])
print("Real label for samples classified with value 1")
display(y_val[preds2[:,0]<preds2[:,1]])

In [ ]:

tok1 = parsing_helpers.create_tokenizer(training_df)
word_index = tok1.word_index
tok1_words = []
for word, i in word_index.items():
    tok1_words.append(word)

for idx, _df in enumerate(gathered_dfs[:-1]):
    tok2 = parsing_helpers.create_tokenizer(_df)
    word_index2 = tok2.word_index

    gathered_f = gathered_data_files[idx]

    tok2_words = []
    for word, i in word_index2.items():
        tok2_words.append(word)

    words_in_tok1 = [w for w in tok2_words if w in tok1_words]
    tok1_words_in_tok2 = [w for w in tok1_words if w in tok2_words]
    print("\n > {}".format(gathered_f))
    print("Gathered source words in training data: {}".format(len(words_in_tok1) / len(tok1_words)))
    print("Training data words in gathered source: {}".format(len(tok1_words_in_tok2) / len(tok2_words)))


In [ ]:

NUM_SENTENCES_TO_SHOW=10

df_test = pd.read_csv(os.path.join(GATHERED_DATA_FOLDER, "gathered_github_sentences.tsv"), sep="\t")

test_sentences = df_test['sentence'].values

test_seq = extended_tokenizer.texts_to_sequences(test_sentences)
test_seq = pad_sequences(val_seq, maxlen=MAX_SEQUENCE_LENGTH)

preds = conv_models[0][CONVENTION].predict(test_seq)[:,1]

print("-----------------------------------------------------")
print(" >> Top sentences with higher confidence for Github:")
print("            ({} convention)".format(CONVENTION))
print("-----------------------------------------------------\n")
for s in test_sentences[preds.argsort()[-NUM_SENTENCES_TO_SHOW:][::-1]]:
    print("{}\n".format(s))
    
    

In [ ]:
### Ading more green examples
green_audit_df = pd.read_csv("Data/Iterative-models-building/Classification results/Conventions/Audited/audited_ALL_2019-10-08-20:23:58.598102.tsv", sep="\t")
green_audit_df = green_audit_df[['text', 'provenance', 'convention', 'new']]
green_audit_df = green_audit_df.rename(columns={"new": "label"})
green_audit_df = green_audit_df[green_audit_df['convention']=='green']
green_audit_df['convention'] = "Green"
green_audit_df.head(5)



In [ ]:
green_df = pd.concat([df, green_audit_df])

In [ ]:
######################
### Cross validation with new data
######################

## 1. Split training, validation data outside the training function
## 2. Train a model with training data
## 3. Check performance (acc) of the model on training and validation
## 4. Repeat num_tests times
## Next cell plots results

## Number of tests = 10
num_tests = 10
num_epochs = 20

accs2 = []
val_accs2 = []
epochs = np.arange(0,num_epochs, 1)

texts = green_df['text'].values
labels = green_df['label'].values
np.random.seed(0)

for n in range(num_tests):
    print("\n\n=================================")
    print("               {}".format(n))
    print("=================================")
    indices = np.arange(len(texts))

    np.random.shuffle(indices)
    texts = texts[indices]
    labels = labels[indices]
    nb_validation_samples = int(VALIDATION_SPLIT * len(texts))

    x_train = texts[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    x_val = texts[-nb_validation_samples:]
    y_val = labels[-nb_validation_samples:]

    df_train = pd.DataFrame(columns=['text', 'label', 'convention'])
    df_train['text'] = x_train
    df_train['label'] = y_train
    df_train['convention'] = CONVENTION


    df_val = pd.DataFrame(columns=['text', 'label', 'convention'])
    df_val['text'] = x_val
    df_val['label'] = y_val
    df_val['convention'] = CONVENTION


    tmp_training = model_helpers.train_DL_models(df_train,
                data_class_column="convention", 
                data_label_column="label",
                df_val=df_val,
                tokenizer=extended_tokenizer,
                random_seed=0,
                use_validation=True, 
                num_epochs = num_epochs)

    accs2.append(tmp_training[4][CONVENTION].history['acc'])
    val_accs2.append(tmp_training[4][CONVENTION].history['val_acc'])       


In [ ]:
fig = plt.figure(figsize=[10,10])
fig.suptitle("Checking performance on multiple random data partitions\n       ({} Convention)".format(CONVENTION), fontsize=16)
plt.subplot(2, 2, 1)
plt.subplots_adjust(wspace=0.4, hspace=0.4)


for i in range(len(accs)):
    plt.plot(epochs,accs2[i], label="Test {}".format(i))
plt.title("Training accuracies per epoch")
plt.xlabel("epochs")
plt.ylabel("acc score")
plt.legend()

plt.subplot(2, 2, 2)
for i in range(len(val_accs)):
    plt.plot(epochs,val_accs2[i], label="Test {}".format(i))
plt.title("Validation accuracies per epoch")
plt.xlabel("epochs")
plt.ylabel("acc score")
plt.legend()
plt.show()

In [ ]:
## Checking performance setting validation data on the model training
## Validation data is created inside the models trainign function
conv_models2 = model_helpers.train_DL_models(green_df,
            data_class_column="convention", 
            data_label_column="label",
            df_val=None,
            tokenizer=extended_tokenizer,
            random_seed=0, 
            use_validation=True)

In [ ]:


NUM_SENTENCES_TO_SHOW=20

df_test = pd.read_csv(os.path.join(GATHERED_DATA_FOLDER, "gathered_github_sentences.tsv"), sep="\t")

test_sentences = df_test['sentence'].values
test_sentences = np.array([sent for sent in test_sentences if len(sent.split(" "))<40])

test_seq = extended_tokenizer.texts_to_sequences(test_sentences)
test_seq = pad_sequences(test_seq, maxlen=MAX_SEQUENCE_LENGTH)

preds = conv_models2[0][CONVENTION].predict(test_seq)[:,1]

print("-----------------------------------------------------")
print(" >> DL model results")
print(" >> Top sentences with higher confidence for Github:")
print("            ({} convention)".format(CONVENTION))
print("-----------------------------------------------------\n")
for s in test_sentences[preds.argsort()[-NUM_SENTENCES_TO_SHOW:][::-1]]:
    print("{}\n".format(s))
    

In [ ]:
_DLModels = conv_models2[0]
_DLTokenizers = conv_models2[1]

In [ ]:
_MLModels = model_helpers.train_ML_models(green_df)

In [ ]:
test_sentences = df_test['sentence'].values

mixture_preds = model_helpers.calculate_matches_mixture(test_sentences, _MLModels, _DLModels, _DLTokenizers)
print("-----------------------------------------------------")
print(" >> Mixture model results")
print(" >> Top sentences with higher confidence for Github:")
print("            ({} convention)".format(CONVENTION))
print("-----------------------------------------------------\n")
for s in test_sentences[mixture_preds[CONVENTION].argsort()[-NUM_SENTENCES_TO_SHOW:][::-1]]:
    print("{}\n".format(s))
    

    